## Example of displaying heatmap using folium (English)  

### Import libraries  

In [1]:
import pandas as pd
import folium

### Parameters

In [2]:
csv_in = 'crime2006_2016_en-utf8.csv'

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

### Read CSV file

In [3]:
df = pd.read_csv(csv_in, delimiter=',', skiprows=0, header=0)
print(df.shape)
print(df.info())
display(df.head())
#display(df)

(517, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Year       517 non-null    int64 
 1   Pref       517 non-null    object
 2   #Arrested  517 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 12.2+ KB
None


,Year,Pref,#Arrested
0,2016,Hokkaido,8460
1,2016,Aomori,1621
2,2016,Iwate,1282
3,2016,Miyagi,3209
4,2016,Akita,1364


In [6]:
df_2016 = df[df['Year']==2016]


### Parameters2 (for lat/lon)   

In [9]:
csv_in2 = 'h3010pref_pos_utf8_en.csv'

### Read CSV file

In [10]:
df_pos = pd.read_csv(csv_in2, delimiter=',', skiprows=0, header=0)
print(df_pos.shape)
display(df_pos.head())

(47, 3)


,Pref,lat,lon
0,Hokkaido,43.064278,141.346785
1,Aomori,40.824338,140.740087
2,Iwate,39.703647,141.152592
3,Miyagi,38.268803,140.871846
4,Akita,39.718058,140.103250


### Merge DataFrames to add lat/lon to pref data  

In [11]:
df_merged = pd.merge(df, df_pos, on='Pref', how='left')
print(df_merged.shape)
display(df_merged.head())

(517, 5)


,Year,Pref,#Arrested,lat,lon
0,2016,Hokkaido,8460,43.064278,141.346785
1,2016,Aomori,1621,40.824338,140.740087
2,2016,Iwate,1282,39.703647,141.152592
3,2016,Miyagi,3209,38.268803,140.871846
4,2016,Akita,1364,39.718058,140.103250


### Make data for heatmap  
[ [lat2, lon2, value2], [lat2, lon2, value2], ... ]  

In [13]:
df_heat = df_merged[['lat', 'lon', '#Arrested']]
print(df_heat.shape)
display(df_heat.head())

(517, 3)


,lat,lon,#Arrested
0,43.064278,141.346785,8460
1,40.824338,140.740087,1621
2,39.703647,141.152592,1282
3,38.268803,140.871846,3209
4,39.718058,140.103250,1364


### Make map  

In [14]:
from folium.plugins import HeatMap

m = folium.Map(location=[35, 140], zoom_start=4)
m.add_child(HeatMap(df_heat, radius=12))
display(m)

### Add color bar  

In [17]:
from folium.plugins import HeatMap
import branca.colormap as cm

m = folium.Map(location=[35, 140], zoom_start=4)
m.add_child(HeatMap(df_heat, radius=12,
                    gradient={0:'blue', 0.5:'lime', 1:'red'}))

min = df_heat['#Arrested'].min()
max = df_heat['#Arrested'].max()
colors = ['blue', 'lime', 'red']
colormap = cm.LinearColormap(colors, vmin=min, vmax=max).to_step(10000)
colormap.caption = 'Number of cases'
m.add_child(colormap)

display(m)